# Setup and Dependencies

In [ ]:
pip install torch torchvision pytorch-lightning efficientnet_pytorch monai scikit-learn pandas numpy matplotlib

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from efficientnet_pytorch import EfficientNet
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from monai.transforms import Compose, Resize, EnsureType
from monai.data import ImageDataset

# Dataset Preparation
- Load lung nodule data (CT scans with labels: benign vs. malignant).
- Apply preprocessing: resizing, normalization, augmentations.
- Define a PyTorch Dataset and DataLoader.

In [ ]:
class LungNoduleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = torch.load(self.image_paths[idx])  # Load preprocessed CT scan tensor
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transforms
transform = Compose([
    Resize((224, 224)),  # Resize to match EfficientNet input size
    EnsureType(),
])

# Load dataset
image_paths = [...]  # List of preprocessed image file paths
labels = [...]  # Corresponding labels

# Train-test split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)

# Create PyTorch datasets
train_dataset = LungNoduleDataset(train_paths, train_labels, transform=transform)
val_dataset = LungNoduleDataset(val_paths, val_labels, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


# Model Definition (EfficientNet + Lightning)
- Use EfficientNet-B0 as the backbone.
- Modify the classifier head for binary classification (nodule malignancy).
- Define the LightningModule for training and validation.

In [ ]:
class LungNoduleClassifier(pl.LightningModule):
    def __init__(self, lr=1e-4):
        super().__init__()
        self.save_hyperparameters()
        
        # Load EfficientNet and modify final layer
        self.model = EfficientNet.from_pretrained("efficientnet-b0")
        # self.model = FusionModel(protocol = 1)  # Custom model for fusion
        self.model._fc = nn.Linear(self.model._fc.in_features, 1)  # Binary classification

        self.loss_fn = nn.BCEWithLogitsLoss()  # For binary classification
        self.lr = lr

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        labels = labels.float().unsqueeze(1)  # Ensure shape compatibility
        logits = self(images)
        loss = self.loss_fn(logits, labels)
        preds = torch.sigmoid(logits).round()
        acc = (preds == labels).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        labels = labels.float().unsqueeze(1)
        logits = self(images)
        loss = self.loss_fn(logits, labels)
        preds = torch.sigmoid(logits).round()
        acc = (preds == labels).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)


# Training and Evaluation

In [ ]:
trainer = pl.Trainer(
    max_epochs=20,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    log_every_n_steps=10
)

# Initialize model
model = LungNoduleClassifier(lr=1e-4)

# Train the model
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)


# Testing and Deployment

In [ ]:
def predict(model, image_path):
    model.eval()
    image = torch.load(image_path)  # Load preprocessed image tensor
    image = transform(image).unsqueeze(0)  # Add batch dimension
    logits = model(image)
    prob = torch.sigmoid(logits).item()
    return "Malignant" if prob > 0.5 else "Benign"

# Example usage
prediction = predict(model, "path/to/sample_ct_scan.pt")
print(f"Prediction: {prediction}")
